In [1]:
import json
import numpy as np
import pandas as pd

import os
from sklearn.externals import joblib


def init(model_path="model.pkl"):
    global model
    model = joblib.load(model_path)

def predict(data):
    try:
        #data = np.array(json.loads(data))
        result = model.predict(data)
        # You can return any data type, as long as it is JSON serializable.
        return result
    except Exception as e:
        error = str(e)
        return error


def predict_proba(data):
    try:
        #data = np.array(json.loads(data))
        result = model.predict_proba(data)
        # You can return any data type, as long as it is JSON serializable.
        return result
    except Exception as e:
        error = str(e)
        return error

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [5]:
id_name = 'Id'
target = "Choice"
model_path = "d_pis_model.pkl"
test_file = "test_d_pis.csv"

init(model_path)

test_df = pd.read_csv(test_file)
y_true = test_df[target]

y_pred = predict(test_df.drop([target],axis=1))

print("accuracy:")
print(np.mean(y_pred==y_true))


from sklearn import metrics

y_pred_prob = predict_proba(test_df.drop([target],axis=1))

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred_prob[:,1], pos_label=1)
print("auc:")
print(metrics.auc(fpr, tpr))

accuracy:
0.8836363636363637
auc:
0.9461423508252222


In [14]:
## predcict test dataset on Kaggle

kaggle_submit_head = ["Id", "Choice"]

kaggle_test_file = "kaggle-test-d_pis.csv"
kaggle_pred_file = "kaggle-test-d_pis-predictions.csv"

kaggle_test_df = pd.read_csv(kaggle_test_file)
print(kaggle_test_df.head())
print(kaggle_test_df.shape)

kaggle_test_id = np.arange(1, kaggle_test_df.shape[0] + 1)

kaggle_y_pred = predict(kaggle_test_df)

   A_follower_count  A_following_count  A_listed_count  A_mentions_received  \
0            614689                400            5729               199.62   
1            739581              10723            1279               102.03   
2              3638               3341              85                 2.79   
3               613               1259              31                 1.33   
4           1352348                331           15647             19129.85   

   A_retweets_received  A_mentions_sent  A_retweets_sent  A_posts  \
0                67.81             9.37             0.36     4.09   
1                96.77             1.54             1.05     3.47   
2                 1.40             2.59             0.60     3.80   
3                 0.36             6.25             2.32     7.49   
4             14552.73             2.07             0.58    16.28   

   A_network_feature_1  A_network_feature_2  ...  B_following_count  \
0                  669                3

In [15]:
print(kaggle_test_id.shape)

assert kaggle_y_pred.shape == kaggle_test_id.shape

kaggle_pred_df = pd.DataFrame({kaggle_submit_head[0]: kaggle_test_id, kaggle_submit_head[1]: kaggle_y_pred})

print(kaggle_pred_df.head())

kaggle_pred_df.to_csv(kaggle_pred_file, index=False)

(5952,)
   Id  Choice
0   1       0
1   2       0
2   3       0
3   4       0
4   5       0
